<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px"> 

#  APIs for financial data - Solutions

---

<br>
For both parts of this lab we will use Python to interact with the Alpha Vantage API and export data from different endpoints to answer two research questions.

# Part 1: GDP and Consumer Sentiment during the pandemic

You have been tasked with analysing how various economic indicators have behaved during the last few years. Your team has already identified a free source of this data: Alpha Vantage (https://www.alphavantage.co). It is now up to you to extract the relevant data about these indicators and write a short report about your findings.

***Note: be sure to sign up for a free API key at https://www.alphavantage.co/support/#api-key (more detailed instructions are in the instruction document for this lab)***

**0. Imports**
Put all Python imports in the cell below. If you later decide you need to import something, you must put it here and re-run!

In [1]:
%pip install --quiet pandas requests    

# pip install --quiet pandas requests is for Jupyter notebooks to install pandas and requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import requests

In [3]:
from io import BytesIO 
# io is for handling byte streams, BytesIO allows us to read bytes as a file-like object

**1. Find the correct API endpoint to retrieve historical data on GDP**

Define a variable `base_url` that is a **string** that is simply the base portion of the Alpha Vantage URL endpoint. That is, if you look at any API call, this is everything before the question mark.

Next, define another variable `API_KEY` that is a **string** and is your Alpha Vantage API key (make one, they're free and you don't need to give a real email!)

Use the [Documentation](https://www.alphavantage.co/documentation/) to help you.

In [4]:
# Load Alpha Vantage key from a simple text file (alpha.key) 

with open("alpha.key", "r", encoding="utf-8") as f:
    API_KEY = f.read().strip()

if not API_KEY:
    raise EnvironmentError("alpha.key is empty. Put only real API key (no quotes) on one line.")

# with open is used to read the API key from a file named alpha.key
# r is for read mode
# encoding="utf-8" ensures the file is read with UTF-8 encoding
# f.read().strip() reads the content of the file and removes any leading/trailing whitespace
# If the API key is empty, an EnvironmentError is raised with a message indicating the issue.

In [5]:
# To Set the base URL 
BASE_URL = "https://www.alphavantage.co/query"

In [6]:
# To perform confirmation (no API key in output)
assert isinstance(API_KEY, str) and API_KEY.strip(), "Missing or empty API key."   

# assert isinstance is a built-in function that checks the type of a variable, 
# in here we check if API_KEY is a non-empty string and raise an error with a message if not.

print("Setup OK")                   # to indicate setup is complete
print("BASE_URL:", BASE_URL)           # print the base URL for verification of setup for next steps to retrieve data from API

Setup OK
BASE_URL: https://www.alphavantage.co/query


In [7]:
# As confirmed from the outputs above, the setup is complete and the base URL is correctly set for Alpha Vantage API.
# I will just provide the BASE_URL and API_KEY variables here for ease of reference for the next steps. 

BASE_URL = "https://www.alphavantage.co/query"
API_KEY = "MY_API_KEY"


**2. Use Python and the `requests` library to make an API call and retrieve historical GDP data at *the highest level of granularity* (i.e. most frequent that the API allows). Remember to add your own API key to the query.**

You should convert the results to JSON so it behaves like a Python dictionary.

In [8]:
# I will call the REAL_GDP endpoint (quarterly) and keep the RAW JSON only
# GDP endpoint (Quarterly) because it's the highest granularity available as per docs.
# I will request JSON (default) so I can inspect the structure before converting.
# I will store the JSON in gdp_json for the next step.
# I will print a tiny summary (keys, record count, first 5 items) to confirm the payload.

import requests

# I will build the request parameters dictionary for the API call
# I am NOT setting datatype=csv here because I want JSON for inspection in Step 3
gdp_params = {
    "function": "REAL_GDP",     # GDP endpoint
    "interval": "quarterly",    # highest granularity available is quarterly as per docs
    "apikey": API_KEY,          # my API key from alpha.key
    }                           

# I will make the API request to get the raw GDP JSON
gdp_resp = requests.get(BASE_URL, params=gdp_params, timeout=30)    # timeout=30 seconds to avoid hanging indefinitely
gdp_resp.raise_for_status()                                         # raise_for_status will raise an error for HTTP errors

# I will parse the JSON
gdp_json = gdp_resp.json()

# I will do a minimal sanity check and a tiny summary printout
if "data" not in gdp_json:
    msg = gdp_json.get("Note") or gdp_json.get("Information") or gdp_json.get("Error Message") or "Unexpected response shape."
    raise RuntimeError(f"Alpha Vantage did not return 'data'. Hint: {msg}")

print("Great! GDP JSON retrieved.")                      # to confirm successful retrieval
print("Top-level keys:", list(gdp_json.keys()))          # to list the top-level keys in the JSON
print("Number of GDP records:", len(gdp_json["data"]))   # to count the number of records in the 'data' key
print("First 5 items (raw):", gdp_json["data"][:5])      # to print the first 5 items in the 'data' key

Great! GDP JSON retrieved.
Top-level keys: ['name', 'interval', 'unit', 'data']
Number of GDP records: 94
First 5 items (raw): [{'date': '2025-04-01', 'value': '5943.384'}, {'date': '2025-01-01', 'value': '5776.724'}, {'date': '2024-10-01', 'value': '5997.184'}, {'date': '2024-07-01', 'value': '5884.567'}, {'date': '2024-04-01', 'value': '5829.384'}]


**3. Convert the data in this JSON to a Pandas dataframe and export to a csv.**

In [9]:
# From the above output, we can confirm that the GDP JSON has been successfully retrieved and contains the expected data.
# I will convert the GDP JSON to a tidy DataFrame, inspect, and save as CSV
# I will turn gdp_json["data"] (list of dicts) into a pandas DataFrame.
# I will clean headers, parse the date and value columns, and sort by date.
# I will inspect columns, first 5 rows, 5 random rows, and dtypes.
# I will save the result as gdp_quarterly.csv for Tableau.

import pandas as pd

# I will make a DataFrame from the JSON payload
gdp_df = pd.DataFrame(gdp_json["data"]).copy()

# I will clean the column names to snake_case for consistency
# I will strip whitespace, convert to lowercase, and replace spaces with underscores
gdp_df.columns = [c.strip().lower().replace(" ", "_") for c in gdp_df.columns]

# I will parse data types (date to datetime, value to numeric)
# I will use errors="coerce" to handle any parsing issues gracefully
gdp_df["date"] = pd.to_datetime(gdp_df["date"], errors="coerce")
gdp_df["value"] = pd.to_numeric(gdp_df["value"], errors="coerce")

# I will sort by date and reset the index for a clean DataFrame
# date sorting ensures chronological order and reset_index(drop=True) removes the old index
gdp_df = gdp_df.sort_values("date").reset_index(drop=True)

# I will inspect the cleaned DataFrame before saving
print("Columns:", list(gdp_df.columns))                        # to list the column names
print("\nFirst 5 rows:")                                       # to print the first 5 rows
print(gdp_df.head(5).to_string(index=False))                   # to display the first 5 rows without index
print("\nRandom 5 rows (to spot oddities):")                   # to print 5 random rows for spot-checking
print(gdp_df.sample(5, random_state=42).to_string(index=False)) # to display 5 random rows without index
print("\nData types (pandas dtypes):")                         # to print the data types of each column
print(gdp_df.dtypes)                                           # to display the data types of each column
print("\nDate range:", gdp_df["date"].min().date(), "→", gdp_df["date"].max().date())    # to show the date range in the data



Columns: ['date', 'value']

First 5 rows:
      date    value
2002-01-01 3501.118
2002-04-01 3608.496
2002-07-01 3650.253
2002-10-01 3712.845
2003-01-01 3582.767

Random 5 rows (to spot oddities):
      date    value
2012-01-01 4279.816
2007-07-01 4208.612
2015-10-01 4799.354
2020-01-01 5055.712
2002-01-01 3501.118

Data types (pandas dtypes):
date     datetime64[ns]
value           float64
dtype: object

Date range: 2002-01-01 → 2025-04-01


In [10]:
# Above inspection looks good and there are no obvious issues.
# Therefore, I will save the cleaned DataFrame to a CSV file 
gdp_df.to_csv("gdp_quarterly.csv", index=False)

# I will print a small confirmation and a tiny preview
print("Good Job! Saved gdp_quarterly.csv")                            # to confirm the file has been saved
print("Rows:", len(gdp_df), "| Columns:", list(gdp_df.columns))  # to print the number of rows and list of columns
print("\nLast 5 rows:")                                          # to print the last 5 rows
print(gdp_df.tail(5).to_string(index=False))                     # to display the last 5 rows without index

Good Job! Saved gdp_quarterly.csv
Rows: 94 | Columns: ['date', 'value']

Last 5 rows:
      date    value
2024-04-01 5829.384
2024-07-01 5884.567
2024-10-01 5997.184
2025-01-01 5776.724
2025-04-01 5943.384


**4. Next, identify the endpoint for finding daily stock price values and query the VXX ticker.**

Make sure to get as much data as possible using the `outputsize` parameter at this endpoint.

VXX is a mutual fund that that adequately represents the [VIX](https://www.investopedia.com/articles/optioninvestor/09/implied-volatility-contrary-indicator.asp#:~:text=VIX%20measures%20the%20market%27s%20expectation%20of%20volatility%20over,trends%20in%20the%20VIX%20can%20inform%20trading%20strategies.) index that represents the fear and volatility in the market. When VIX (or VXX) is high, fear controls the market, and when VIX (or VXX) is low, people have more confidence in the market.

In [11]:
# I will fetch VXX (daily) and only inspect what came back
# I will request CSV and load it into pandas.
# I will list all columns, show 5 example rows, and print data types.
# I will NOT drop/convert/save yet; this step is just to understand the raw shape.

# I will build the request parameters for the API call
params = {
    "function": "TIME_SERIES_DAILY",  # This is the daily time series endpoint for daily stock/ETF prices
    "symbol": "VXX",                  # I am querying the VXX ETN
    "outputsize": "full",             # I want the full available history
    "datatype": "csv",                # I want the response in CSV format
    "apikey": API_KEY,                # my API key loaded earlier from alpha.key
}

# I will make the API request and load the CSV into a DataFrame
resp = requests.get(BASE_URL, params=params, timeout=30)
vxx_raw = pd.read_csv(BytesIO(resp.content))

# I will inspect structure before any cleaning
print("Columns:", list(vxx_raw.columns))                          # to list all column names
print("\nFirst 5 rows:")                                          # to label the preview of the first 5 rows
print(vxx_raw.head(5).to_string(index=False))                     # to print the first 5 rows of the DataFrame without the index
print("\nRandom 5 rows (to spot oddities):")                      # to label the preview of 5 random rows
print(vxx_raw.sample(5, random_state=42).to_string(index=False))  # to print 5 random rows of the DataFrame without the index for spotting any oddities, using random_state for reproducibility
print("\nData types (pandas dtypes):")                            # to label the data types output
print(vxx_raw.dtypes)                                             # to print the data types of each column in the DataFrame

Columns: ['timestamp', 'open', 'high', 'low', 'close', 'volume']

First 5 rows:
 timestamp  open   high    low  close   volume
2025-10-17 39.22 40.825 36.770  36.95 19695614
2025-10-16 36.22 40.240 36.075  39.97 26356261
2025-10-15 35.27 37.468 35.010  36.58 10753821
2025-10-14 36.59 37.815 34.980  36.15 13616518
2025-10-13 35.36 35.690 34.410  34.62 11007351

Random 5 rows (to spot oddities):
 timestamp  open   high   low  close   volume
2023-02-07 11.69  11.71 11.14  11.24  8694257
2018-07-06 36.03  36.17 34.15  34.23 34722772
2010-12-13 37.63  38.91 37.59  38.69  9328000
2010-12-10 37.96  38.50 37.73  38.13  8243200
2009-04-20 94.59 100.12 93.84  99.59   646400

Data types (pandas dtypes):
timestamp     object
open         float64
high         float64
low          float64
close        float64
volume         int64
dtype: object


**5. Cast the result as a dataframe and export it to a csv.**

In [12]:
# From my inspection step, I noticed all columns are required since they contain price/volume data.
# Hence, I will keep all 6 columns: timestamp, open, high, low, close, volume.
# I will take vxx_raw from Step 4 and finish a tidy version for analysis steps and visualization in Tableau later.
# I will copy vxx_raw so I don't accidentally change the original inspection DataFrame.
# I will convert timestamp to a real date and price/volume columns to numeric.
# I will drop duplicate dates (safety), sort oldest to the newest, and save to vxx_daily.csv.

# I will make a working copy of the raw DataFrame from step 4
vxx = vxx_raw[["timestamp", "open", "high", "low", "close", "volume"]].copy()

# I will convert the date and numeric columns
vxx["timestamp"] = pd.to_datetime(vxx["timestamp"], errors="coerce") 
# I parse the date column by making timestamp a datetime type, errors="coerce" will set invalid parsing as NaT

for col in ["open", "high", "low", "close", "volume"]:
    vxx[col] = pd.to_numeric(vxx[col], errors="coerce")
# I convert price/volume columns to numeric, errors="coerce" will set invalid parsing as NaN

# I will drop exact duplicate dates (just in case) and sort by date
vxx = (
    vxx.drop_duplicates(subset=["timestamp"])    # drop duplicate dates based on timestamp
       .sort_values("timestamp")                 # sort by timestamp from oldest to newest
       .reset_index(drop=True)                   # reset index after sorting, drop=True to avoid adding old index as a column
)

# I will save the cleaned DataFrame to a CSV file 
vxx.to_csv("vxx_daily.csv", index=False)         # save to vxx_daily.csv without the index column

# I will print a small summary and preview
print("Good job! Saved vxx_daily.csv")                                  # to confirm the file has been saved
print("Rows:", len(vxx), "| Columns:", list(vxx.columns))               # to print the number of rows and list of columns
print("Date range:", vxx["timestamp"].min().date(), "to", vxx["timestamp"].max().date())   # to print the date range in the data
print("\nLast 5 rows:")                                                 # to label the preview of the last 5 rows
print(vxx.tail(5).to_string(index=False))                               # to display the last 5 rows without index

Good job! Saved vxx_daily.csv
Rows: 4206 | Columns: ['timestamp', 'open', 'high', 'low', 'close', 'volume']
Date range: 2009-01-30 to 2025-10-17

Last 5 rows:
 timestamp  open   high    low  close   volume
2025-10-13 35.36 35.690 34.410  34.62 11007351
2025-10-14 36.59 37.815 34.980  36.15 13616518
2025-10-15 35.27 37.468 35.010  36.58 10753821
2025-10-16 36.22 40.240 36.075  39.97 26356261
2025-10-17 39.22 40.825 36.770  36.95 19695614


**6. Repeat steps 4 and 5, but this time for VTI**

Use all of the same settings to now gather data for VTI, a mutual fund that consists of _every_ US stock. VTI can be considered an indicator for the overall market.

Again, as in step 5, save this data out to a csv.

In [13]:
# I will break Step 6 into two parts for clarity (step 6.1 and 6.2)
# This will be Step 6.1 and I will perform the inspection only.
# I will fetch VTI (daily) and only inspect what came back
# I will use TIME_SERIES_DAILY with outputsize="full".
# I will request CSV so it loads easily into pandas.
# I will inspect columns, first 5 rows, random 5 rows, and dtypes.
# I will NOT clean or save here; that will be the next step 6.2.


# I will build the request parameters for the API call
vti_params = {
    "function": "TIME_SERIES_DAILY",  # To request time series daily for stock/ETF price data
    "symbol": "VTI",                  # To request data for the VTI ETF
    "outputsize": "full",             # To get as much history as possible
    "datatype": "csv",                # To get CSV for easy loading
    "apikey": API_KEY,                # my API key from alpha.key
}

# I will make the API request and load the CSV into a DataFrame
vti_resp = requests.get(BASE_URL, params=vti_params, timeout=30)
# To avoid hanging indefinitely, I set timeout=30 seconds
vti_raw = pd.read_csv(BytesIO(vti_resp.content))
# To load the CSV content into a pandas DataFrame 

# I will inspect the raw DataFrame before any cleaning
print("Columns:", list(vti_raw.columns))                       # to list the column names
print("\nFirst 5 rows:")                                       # to print the first 5 rows
print(vti_raw.head(5).to_string(index=False))                  # to display the first 5 rows without index
print("\nRandom 5 rows (to spot oddities):")                   # to print 5 random rows for spot-checking
print(vti_raw.sample(5, random_state=42).to_string(index=False))  # to display 5 random rows without index
print("\nData types (pandas dtypes):")                         # to print the data types of each column
print(vti_raw.dtypes)                                          # to display the data types of each column

Columns: ['timestamp', 'open', 'high', 'low', 'close', 'volume']

First 5 rows:
 timestamp    open    high      low  close  volume
2025-10-17 325.010 327.940 324.2400 327.30 5646772
2025-10-16 329.370 329.935 324.0800 325.77 5328475
2025-10-15 329.390 330.990 325.2950 328.38 4245557
2025-10-14 323.900 328.620 322.0800 326.83 6307297
2025-10-13 325.785 327.580 325.1632 326.93 4476790

Random 5 rows (to spot oddities):
 timestamp   open    high    low  close  volume
2010-12-22  64.81  65.020  64.77  64.93 2097600
2021-01-08 199.14 199.460 196.98 199.25 7538654
2005-04-15 112.73 112.930 111.08 111.10  223800
2023-10-18 215.11 215.630 212.59 213.09 3254961
2023-02-07 206.00 209.563 205.03 208.88 3028103

Data types (pandas dtypes):
timestamp     object
open         float64
high         float64
low          float64
close        float64
volume         int64
dtype: object


In [14]:
# From my inspection step 6.1, I noticed all columns are required since they contain price/volume data.
# Hence, I will keep all 6 columns: timestamp, open, high, low, close, volume.
# I will tidy VTI and save a clean CSV for analysis and visualization later.
# I will convert timestamp to datetime and prices/volume to numeric.
# I will drop duplicate dates (safety), sort oldest to newest, and save vti_daily.csv.
# I will print a short summary and a tiny preview.

# I will make a working copy of the raw DataFrame from above step 6.1.
vti = vti_raw[["timestamp", "open", "high", "low", "close", "volume"]].copy()

# I will convert the date and numeric columns
vti["timestamp"] = pd.to_datetime(vti["timestamp"], errors="coerce")
# I parse the date column by making timestamp a datetime type, errors="coerce" will set invalid parsing as NaT

for col in ["open", "high", "low", "close", "volume"]:
    vti[col] = pd.to_numeric(vti[col], errors="coerce")
# I convert price/volume columns to numeric, errors="coerce" will set invalid parsing as NaN

# I will drop exact duplicate dates and sort by date
vti = (
    vti.drop_duplicates(subset=["timestamp"])   # drop duplicate dates based on timestamp
       .sort_values("timestamp")                # sort by timestamp from oldest to newest
       .reset_index(drop=True)                  # reset index after sorting, drop=True to avoid adding old index as a column
)

# I will save the cleaned DataFrame to a CSV file
vti.to_csv("vti_daily.csv", index=False)      # save to vti_daily.csv without the index column

# I will print a small summary and preview
print("Good Job! Saved vti_daily.csv")                                     # to confirm the file has been saved
print("Rows:", len(vti), "| Columns:", list(vti.columns))           # to print the number of rows and list of columns
print("Date range:", vti["timestamp"].min().date(), "→", vti["timestamp"].max().date())  # to print the date range in the data
print("\nLast 5 rows:")                                             # to label the preview of the last 5 rows
print(vti.tail(5).to_string(index=False))                           # to display the last 5 rows without index

Good Job! Saved vti_daily.csv
Rows: 6133 | Columns: ['timestamp', 'open', 'high', 'low', 'close', 'volume']
Date range: 2001-05-31 → 2025-10-17

Last 5 rows:
 timestamp    open    high      low  close  volume
2025-10-13 325.785 327.580 325.1632 326.93 4476790
2025-10-14 323.900 328.620 322.0800 326.83 6307297
2025-10-15 329.390 330.990 325.2950 328.38 4245557
2025-10-16 329.370 329.935 324.0800 325.77 5328475
2025-10-17 325.010 327.940 324.2400 327.30 5646772


# Part 2: Cryptocurrencies

Your stakeholders are becoming aware of the rise in cryptocurrencies, and would like to understand the recent growth of this market. Your task is to use the Alpha Vantage API to extract historical data on cryptocurrency market performance, and tell a story about their growth using visuals created in a BI tool of your choice (Tableau/Power BI).

**1. Find the correct API endpoints to retrieve historical data on cryptocurrency prices over time. Daily should be a sufficient level of granularity for your purposes.**

Use any specific cryptocurrency you wish (e.g. Bitcoin) against the US Dollar.

Use the [Documentation](https://www.alphavantage.co/documentation/) to help you.

**2. Use Python to read the data as JSON**

In [15]:
# My cryptocurrency of choice is Bitcoin (BTC) for this project.
# I choose BTC because it is the most well-known and widely used cryptocurrency with high liquidity and it has a rich historical data set.
# It is also often used as a benchmark for the overall cryptocurrency market.
# In this step, I will only fetch the RAW JSON for BTC weekly in USD and quickly inspect the structure.
# I am fetching BTC weekly compared to daily because daily data only has free tier returns ~ 365 days for BTC-USD as per Alpha Vantage documentation.
# To get a few years of data, weekly is more suitable as Alpha Vantage provides more historical data points for weekly intervals.

# I will separate Step 2 into two parts for clarity.
# Step 2.1 will fetch BTC JSON only for inspection in Step 2.2.
# I will keep the JSON in btcw_json for Step 2.2 (deeper inspection) and Step 3 (export).

# Part 2 — Step 2.1 

import requests

# I will build the request parameters for the API call (weekly crypto)
btcw_params = {
    "function": "DIGITAL_CURRENCY_WEEKLY",  # to get weekly digital currency data for Bitcoin (Crypto endpoint)
    "symbol": "BTC",                        # BTC = Bitcoin
    "market": "USD",                        # quote currency = US Dollar
    "apikey": API_KEY,                      # my API key from alpha.key
}

# I will make the API request to get the raw BTC weekly JSON
btcw_resp = requests.get(BASE_URL, params=btcw_params, timeout=30)  # timeout=30 seconds to avoid hanging indefinitely
btcw_resp.raise_for_status()                 # raise_for_status will raise an error for HTTP errors
btcw_json = btcw_resp.json()                  # btcw_resp.json() parses the JSON response into a Python dictionary for further processing


# I will print quick confirmation so I know what I received and can inspect next
print("Great! BTC WEEKLY JSON retrieved.")             # to confirm successful retrieval of BTC weekly JSON


Great! BTC WEEKLY JSON retrieved.


In [16]:
# Still Step 2.1 — just confirming what keys exist as previous steps do not provide this info

print("Top-level key count:", len(btcw_json))          # to count the number of top-level keys in the JSON for quick overview
print("Top-level keys:", list(btcw_json.keys()))       # to list all the top-level keys in the JSON

# I will locate the time-series key and peek at the newest date + its fields
ts_key_w = "Time Series (Digital Currency Weekly)"          # key for weekly digital currency time series data
if ts_key_w not in btcw_json:
    raise KeyError(f"Expected key not found: {ts_key_w}")     # raise KeyError if the expected time-series key is not found in the JSON

latest_date = sorted(btcw_json[ts_key_w].keys(), reverse=True)[0]   # to get the latest date by sorting the keys in descending order and taking the first one
print("\nLatest weekly date in JSON:", latest_date)                 # to print the latest weekly date available in the JSON time series data
print("Fields on that date:", list(btcw_json[ts_key_w][latest_date].keys()))  # to print the field names for the latest date in the time series data


Top-level key count: 2
Top-level keys: ['Meta Data', 'Time Series (Digital Currency Weekly)']

Latest weekly date in JSON: 2025-10-19
Fields on that date: ['1. open', '2. high', '3. low', '4. close', '5. volume']


In [17]:
# Above output confirms I have the expected structure and fields for BTC weekly JSON.
# Now, Step 2.2 — I will do a deeper inspection of the BTC weekly JSON.
# I will turn the weekly BTC JSON (btcw_json) into a quick pandas DataFrame for inspection.
# I will not clean or save here; I just want to confirm shape, columns, dtypes, and a string date range.

import pandas as pd


# I will extract the time series data from the JSON
ts = btcw_json[ts_key_w]  # dict: {'YYYY-MM-DD': {'1. open': '...', '2. high': '...'} ...}

# I will build records using safe lookups (fields present in weekly payload)
def pick(row, *candidates):
    # I will return the first available field among the candidates
    for c in candidates:
        if c in row:
            return row[c]
    return None

records = []
for date_str, row in ts.items():
    records.append({
        "date": date_str,                       # keep string for now (I will type it when cleaning)
        "open":   pick(row, "1. open"),         # open price
        "high":   pick(row, "2. high"),         # high price
        "low":    pick(row, "3. low"),          # low price
        "close":  pick(row, "4. close"),        # close price
        "volume": pick(row, "5. volume"),       # reported volume (note: crypto volume units vary by source)
    })

btcw_df = pd.DataFrame.from_records(records)

# I will do a light inspection (columns, first 5, random 5, dtypes, null counts, string date range)
print("Columns:", list(btcw_df.columns))
print("\nFirst 5 rows:")                        # to label the preview of the first 5 rows
print(btcw_df.head(5).to_string(index=False))   # to display the first 5 rows without index

print("\nRandom 5 rows (to spot oddities):")                        # to label the preview of 5 random rows
print(btcw_df.sample(5, random_state=42).to_string(index=False))    # to display 5 random rows without index

print("\nData types (pandas dtypes):")                 # to label the data types output
print(btcw_df.dtypes)                                  # to print the data types of each column

print("\nNulls by column (before cleaning):")        # to label the null counts output
print(btcw_df.isna().sum())                          # to print the count of null values in each column

# I will also print the string date range (typed dates will be in the cleaning step)
dates = sorted(btcw_df["date"])
print("\nString date range:", dates[0], "to", dates[-1])    # to print the string date range from earliest to latest
print("Rows:", len(btcw_df))                                # to print the total number of rows in the DataFrame

Columns: ['date', 'open', 'high', 'low', 'close', 'volume']

First 5 rows:
      date            open            high             low           close         volume
2025-10-19 115067.96000000 116077.51000000 103516.75000000 107035.91000000 59819.95219502
2025-10-12 123520.80000000 126296.00000000 107000.00000000 115067.98000000 73716.51126655
2025-10-05 112197.66000000 125750.00000000 111582.65000000 123520.79000000 49462.68044524
2025-09-28 115282.26000000 115441.37000000 108623.70000000 112197.67000000 38310.34046273
2025-09-21 115314.12000000 117998.17000000 114395.84000000 115282.27000000 37222.96619062

Random 5 rows (to spot oddities):
      date            open            high             low           close          volume
2025-07-20 119130.81000000 123231.07000000 115697.37000000 117312.70000000  67814.54474915
2024-12-08  97259.17000000 104000.00000000  92055.86000000 101174.99000000 114187.97430796
2024-11-24  89877.11000000  99860.00000000  89372.65000000  98028.18000000 13

In [18]:
# Above inspection looks good for BTC weekly JSON converted to DataFrame shape 
# But I noticed that I only got approximately 1 year of weekly data (49 rows) from 2024-11-17 to 2025-10-19 due to free tier limitations.

# Therefore, I will now proceed to Part 2 — Step 2.3 to check whether I could fetch BTC monthly JSON for a longer history.
# Part 2 — Step 2.3 (Monthly)
# I will fetch Bitcoin (BTC) monthly data quoted in USD as RAW JSON only.
# I will keep the JSON in btcm_json for the next steps (inspection, then export).

import requests

# I will build the request parameters for the API call (monthly crypto)
btcm_params = {
    "function": "DIGITAL_CURRENCY_MONTHLY",   # to get monthly digital currency data for Bitcoin
    "symbol": "BTC",                           # BTC = Bitcoin
    "market": "USD",                           # quote currency = US Dollar
    "apikey": API_KEY,                         # my API key from alpha.key
}

# I will make the API request to get the raw BTC monthly JSON
btcm_resp = requests.get(BASE_URL, params=btcm_params, timeout=30)  # timeout=30 seconds
btcm_resp.raise_for_status()                                         # raise error for HTTP issues
btcm_json = btcm_resp.json()                                         # parse JSON into a Python dict

# I will print a tiny confirmation so I know what I received and can inspect next
print("Great! BTC MONTHLY JSON retrieved.")
print("Top-level key count:", len(btcm_json))
print("Top-level keys (first 5):", list(btcm_json.keys())[:5])

# I will locate the time-series key and peek at the newest date + its fields
ts_key_m = "Time Series (Digital Currency Monthly)"
if ts_key_m not in btcm_json:
    raise KeyError(f"Expected key not found: {ts_key_m}")

latest_date_m = sorted(btcm_json[ts_key_m].keys(), reverse=True)[0]   # newest month
print("\nLatest monthly date in JSON:", latest_date_m)
print("Fields on that date:", list(btcm_json[ts_key_m][latest_date_m].keys()))

Great! BTC MONTHLY JSON retrieved.
Top-level key count: 2
Top-level keys (first 5): ['Meta Data', 'Time Series (Digital Currency Monthly)']

Latest monthly date in JSON: 2025-10-19
Fields on that date: ['1. open', '2. high', '3. low', '4. close', '5. volume']


In [19]:
# From the above output, I can confirm that I have successfully retrieved the BTC monthly JSON and it contains the expected structure and fields.
# Now, Step 2.4 — I will confirm the BTC MONTHLY history span (earliest month/year, latest month, and count)

# I will grab the monthly time series mapping (YYYY-MM-DD -> fields dict)
ts_m = btcm_json[ts_key_m]

# I will sort keys ascending to find the earliest month
all_months = sorted(ts_m.keys())                 # ascending chronological
earliest_date_m = all_months[0]                  # earliest month
latest_date_m   = all_months[-1]                 # latest month
n_months        = len(all_months)                # total number of monthly records

# I will print a concise summary
print("Records (months):", n_months)
print("Date range:", earliest_date_m, "to", latest_date_m)

# I will also peek at the earliest month’s fields to confirm structure
print("\nEarliest month fields:", list(ts_m[earliest_date_m].keys()))
print("Earliest month values:", ts_m[earliest_date_m]) 

Records (months): 11
Date range: 2024-12-31 to 2025-10-19

Earliest month fields: ['1. open', '2. high', '3. low', '4. close', '5. volume']
Earliest month values: {'1. open': '96464.95000000', '2. high': '108388.88000000', '3. low': '91271.19000000', '4. close': '93354.22000000', '5. volume': '422201.07415397'}


In [20]:
# Step 2.5 — Decision & scope for crypto
# After testing the weekly and monthly crypto endpoints, I confirmed Alpha Vantage free tier returns ~1 year of history for crypto. 
# Daily is the richest of the three, so I will proceed with BTC DAILY and clearly state this limitation in the report.
# For fair comparison with VTI and VXX later, I will align to the common last ~365 days.
# (The exact window will be derived after we load BTC daily, VTI daily, and VXX daily.)

print("Step 2.5 OK — proceeding with BTC DAILY (~1 year, free tier).")

Step 2.5 OK — proceeding with BTC DAILY (~1 year, free tier).


In [21]:
# Step 2.6 — I will fetch BTC DAILY (JSON only) and do a quick sanity check

import requests

btcd_params = {
    "function": "DIGITAL_CURRENCY_DAILY",   # daily digital currency endpoint
    "symbol":   "BTC",                       # BTC = Bitcoin
    "market":   "USD",                       # quote currency in USD
    "apikey":   API_KEY,                     # my Alpha Vantage API key (loaded earlier)
}

btcd_resp = requests.get(BASE_URL, params=btcd_params, timeout=30)
btcd_resp.raise_for_status()                 # raise error for HTTP issues
btcd_json = btcd_resp.json()                 # parse JSON into Python dictionary for further processing

print("Great! BTC DAILY JSON retrieved.")                    # to confirm successful retrieval
print("Top-level key count:", len(btcd_json))                # to count the number of top-level keys
print("Top-level keys (first 5):", list(btcd_json.keys())[:5])  # to list the first 5 top-level keys

# locate the time-series key and peek at latest date + fields
ts_key_d = "Time Series (Digital Currency Daily)"       # time-series key for daily digital currency data
if ts_key_d not in btcd_json:                    
    raise KeyError(f"Expected key not found: {ts_key_d}")  # to raise error if expected key is missing

latest_date_d = sorted(btcd_json[ts_key_d].keys(), reverse=True)[0]
print("\nLatest date in JSON:", latest_date_d)                                 # to print the latest date available in the JSON time series data
print("Fields on that date:", list(btcd_json[ts_key_d][latest_date_d].keys())) # to print the field names for the latest date in the time series data

Great! BTC DAILY JSON retrieved.
Top-level key count: 2
Top-level keys (first 5): ['Meta Data', 'Time Series (Digital Currency Daily)']

Latest date in JSON: 2025-10-19
Fields on that date: ['1. open', '2. high', '3. low', '4. close', '5. volume']


**3. Identify the key which holds the data itself and export it as a csv.**

In [22]:
# I will separate Step 3 into two parts for clarity : Step 3.1 and Step 3.2.
# In Step 3.1, I will build an inspection DataFrame (strings) to confirm shape & ranges.

import pandas as pd

ts = btcd_json[ts_key_d]   # dict: 'YYYY-MM-DD' -> { '1. open', '2. high', '3. low', '4. close', '5. volume', '6. market cap (USD)' }

def pick(row, *candidates):
    # returns the first field that exists (handles variants like '1a. open (USD)')
    for c in candidates:
        if c in row:
            return row[c]
    return None

records = []
for date_str, row in ts.items():
    records.append({
        "date":           date_str,                                  # keep as string now; will convert when cleaning 
        "open_usd":       pick(row, "1a. open (USD)", "1. open"),    
        "high_usd":       pick(row, "2a. high (USD)", "2. high"),
        "low_usd":        pick(row, "3a. low (USD)",  "3. low"),
        "close_usd":      pick(row, "4a. close (USD)","4. close"),
        "volume":         pick(row, "5. volume"),
        "market_cap_usd": pick(row, "6. market cap (USD)"),
    })

btc_df = pd.DataFrame.from_records(records)

# light inspection
print("Columns:", list(btc_df.columns))                     # to list the column names
print("\nFirst 5 rows:")                                    # to label the preview of the first 5 rows
print(btc_df.head(5).to_string(index=False))                # to display the first 5 rows without index

print("\nRandom 5 rows (to spot oddities):")                # to label the preview of 5 random rows
print(btc_df.sample(5, random_state=42).to_string(index=False))  # to display 5 random rows without index

print("\nData types (pandas dtypes):")                      # to label the data types output
print(btc_df.dtypes)                                        # to print the data types of each column
  
print("\nNulls by column (before cleaning):")                 # to label the null counts output
print(btc_df.isna().sum())                                    # to print the count of null values in each column

# string date range
dates = sorted(btc_df["date"])                                 # sort date strings 
print("\nString date range:", dates[0], "to", dates[-1])       # to print the string date range from earliest to latest
print("Rows:", len(btc_df))                                    # to print the total number of rows in the DataFrame

Columns: ['date', 'open_usd', 'high_usd', 'low_usd', 'close_usd', 'volume', 'market_cap_usd']

First 5 rows:
      date        open_usd        high_usd         low_usd       close_usd         volume market_cap_usd
2025-10-19 107208.91000000 107300.00000000 107000.00000000 107035.91000000   107.05717453           None
2025-10-18 106462.09000000 107535.68000000 106357.69000000 107208.91000000  3552.60959467           None
2025-10-17 108197.99000000 109269.97000000 103516.75000000 106463.30000000 13824.34463474           None
2025-10-16 110804.11000000 112021.07000000 107464.53000000 108198.00000000 14792.78754529           None
2025-10-15 113072.18000000 113671.99000000 110185.87000000 110804.12000000  7440.03782479           None

Random 5 rows (to spot oddities):
      date        open_usd        high_usd         low_usd       close_usd         volume market_cap_usd
2025-05-15 103540.91000000 104200.00000000 101400.01000000 103786.43000000  7563.00946962           None
2024-11-12  8877

In [23]:
# From the above inspection, I can confirm that the BTC daily JSON has been successfully converted to a DataFrame and contains the expected structure and fields.
# Now, Step 3.2 — I will clean, standardize, and export the BTC DAILY dataset for analysis and Tableau visualization later.

# I will make a copy so the raw inspection df remains untouched
btc = btc_df.copy()

# Rename to align with VTI/VXX style (drop "_usd" suffix for OHLC)
btc = btc.rename(columns={
    "open_usd":  "open",
    "high_usd":  "high",
    "low_usd":   "low",
    "close_usd": "close",
})
# 'volume' and 'market_cap_usd' already have good names hence no renaming needed

# Parse date and cast numeric columns
btc["date"] = pd.to_datetime(btc["date"], errors="coerce") 
  # I parse the date column by making 'date' a datetime type, errors="coerce" will set invalid parsing as NaT
for col in ["open", "high", "low", "close", "volume", "market_cap_usd"]:
    if col in btc.columns:
        btc[col] = pd.to_numeric(btc[col], errors="coerce")

# Drop exact duplicate dates (safety), sort oldest→newest, reset index
btc = (
    btc.drop_duplicates(subset=["date"])
       .sort_values("date")
       .reset_index(drop=True)
)

# Export (no index)
btc.to_csv("btc_daily.csv", index=False)

# Confirmation + tiny preview
print("Good job! Saved btc_daily.csv")
print("Rows:", len(btc), "| Columns:", list(btc.columns))
print("Date range:", btc["date"].min().date(), "→", btc["date"].max().date())
print("\nLast 5 rows:")
print(btc.tail(5).to_string(index=False))

Good job! Saved btc_daily.csv
Rows: 350 | Columns: ['date', 'open', 'high', 'low', 'close', 'volume', 'market_cap_usd']
Date range: 2024-11-04 → 2025-10-19

Last 5 rows:
      date      open      high       low     close       volume  market_cap_usd
2025-10-15 113072.18 113671.99 110185.87 110804.12  7440.037825             NaN
2025-10-16 110804.11 112021.07 107464.53 108198.00 14792.787545             NaN
2025-10-17 108197.99 109269.97 103516.75 106463.30 13824.344635             NaN
2025-10-18 106462.09 107535.68 106357.69 107208.91  3552.609595             NaN
2025-10-19 107208.91 107300.00 107000.00 107035.91   107.057175             NaN


In [24]:
# Finally, CSV file (btc_daily.csv) is saved and ready for analysis and visualization in Tableau later.
# Quick sanity check for expected columns and row count to catch any major issues before analysis
expected = {"date","open","high","low","close","volume"}
missing = expected - set(btc.columns)
if missing:
    print("⚠️ Missing expected columns:", missing)
if len(btc) < 300:
    print(f"⚠️ Row count lower than expected: {len(btc)}")



### ✅ Data Acquisition Complete

I’ve finished Part 1 (Economic Indicators) and Part 2 (Cryptocurrencies) data collection, inspection, and export.
All raw calls were inspected first (JSON/CSV preview, columns, sample rows, dtypes), then converted to tidy DataFrames and saved as CSV for downstream analysis/visualization.

Files exported are:

 (1) gdp_quarterly.csv — Real GDP (quarterly), 2002-01-01 → 2025-04-01, 2 columns (date, value)

 (2) vti_daily.csv — VTI daily OHLCV, 2001-05-31 → 2025-10-17, 6 columns

 (3) vxx_daily.csv — VXX daily OHLCV, 2009-01-30 → 2025-10-17, 6 columns

 (4) btc_daily.csv — BTC-USD daily OHLCV, ~1 year (2024-11-04 → 2025-10-19), 7 columns (market_cap_usd not populated by free tier; documented)

#### Note on BTC history: 

For Bitcoin specifically, the free Alpha Vantage daily crypto endpoint provides roughly one year of history; I tested weekly and monthly endpoints but they returned even shorter spans in my environment, so I proceeded with daily BTC and documented this limitation.

#### New Notebook: Wrangling, Analysis, Visuals
I will rename this current notebook as 6.06 APIs for Financial Data_PartA_Data Retrieval.ipynb. 

Next, I will continue the project in a fresh notebook named 6.06_APIs for Finanical Data_PartB_Analysis_Wrangling_Visuals.ipynb. 

In that notebook I will load the four CSVs, perform light wrangling (date parsing, alignment, de-duplication), and create analysis fields such as returns and rolling metrics. I will then run the core analyses for Part 1 and Part 2, comparing trends (e.g., BTC vs VTI and VXX) and writing concise Markdown notes under each section to explain cleaning choices, assumptions, and findings.

I will produce Python charts to support the narrative, using clear captions that summarize the key insight for each figure. 

#### Tableau Companion
I’ll also build a small Tableau workbook using the same CSVs to produce a few supporting visuals.

Each chart will include a concise caption with the key takeaway consistent with the notebook analysis.

#### Final report (Word)

Finally, I will summarize the wrangling steps, the main results for both parts, and the best charts in the provided Word document 6.06. Alpha Vantage data lab worksheet.docx. This closes the data collection phase and sets up a clean path for analysis and storytelling.